In [1]:
import sys
from pyspark.sql import SparkSession, functions, types

spark = SparkSession.builder.appName('reddit averages').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

assert sys.version_info >= (3, 8) # make sure we have Python 3.8+
assert spark.version >= '3.2' # make sure we have Spark 3.2+


comments_schema = types.StructType([
    types.StructField('archived', types.BooleanType()),
    types.StructField('author', types.StringType()),
    types.StructField('author_flair_css_class', types.StringType()),
    types.StructField('author_flair_text', types.StringType()),
    types.StructField('body', types.StringType()),
    types.StructField('controversiality', types.LongType()),
    types.StructField('created_utc', types.StringType()),
    types.StructField('distinguished', types.StringType()),
    types.StructField('downs', types.LongType()),
    types.StructField('edited', types.StringType()),
    types.StructField('gilded', types.LongType()),
    types.StructField('id', types.StringType()),
    types.StructField('link_id', types.StringType()),
    types.StructField('name', types.StringType()),
    types.StructField('parent_id', types.StringType()),
    types.StructField('retrieved_on', types.LongType()),
    types.StructField('score', types.LongType()),
    types.StructField('score_hidden', types.BooleanType()),
    types.StructField('subreddit', types.StringType()),
    types.StructField('subreddit_id', types.StringType()),
    types.StructField('ups', types.LongType()),
    #types.StructField('year', types.IntegerType()),
    #types.StructField('month', types.IntegerType()),
])


def main(in_directory, out_directory):
    comments = spark.read.json(in_directory, schema=comments_schema)

    # TODO: calculate averages, sort by subreddit. Sort by average score and output that too.

    

    #averages_by_subreddit.write.csv(out_directory + '-subreddit', mode='overwrite')
    #averages_by_score.write.csv(out_directory + '-score', mode='overwrite')


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/15 11:26:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
comments = spark.read.json('reddit-1', schema=comments_schema)

averages = comments.groupBy(comments['subreddit']).avg('score')
# averages = averages.agg(functions.avg(comments['score']))
averages.show()

+---------+------------------+
|subreddit|        avg(score)|
+---------+------------------+
|     xkcd| 5.272939881689366|
|    scala| 1.928939237899073|
|optometry|1.4701986754966887|
|  Cameras|1.2222222222222223|
|Genealogy| 1.871313672922252|
+---------+------------------+



In [17]:
averages_by_subreddit = averages.sort(["subreddit"], ascending=[True])
averages_by_subreddit.show()

+---------+------------------+
|subreddit|        avg(score)|
+---------+------------------+
|  Cameras|1.2222222222222223|
|Genealogy| 1.871313672922252|
|optometry|1.4701986754966887|
|    scala| 1.928939237899073|
|     xkcd| 5.272939881689366|
+---------+------------------+



In [15]:
averages_by_score = averages.sort(["avg(score)"], ascending=[False])
averages_by_score.show()

+---------+------------------+
|subreddit|        avg(score)|
+---------+------------------+
|     xkcd| 5.272939881689366|
|    scala| 1.928939237899073|
|Genealogy| 1.871313672922252|
|optometry|1.4701986754966887|
|  Cameras|1.2222222222222223|
+---------+------------------+

